In [38]:
import tensorflow as tf
from elmo import elmo_embedding
import numpy as np
import os
from tqdm import tqdm
import json
from model_v2 import LanguageModel

In [2]:
with open('pos2idx.json', 'r') as inp:
    pos2idx = json.load(inp)
with open('ner2idx.json', 'r') as inp:
    ner2idx = json.load(inp)

In [3]:
def tagger(inputs, labels, seq_lens, n_units, n_classes, drop_i, drop_o, name='tagger', reuse=False, is_training=True):
    with tf.variable_scope(name, reuse=reuse):
        def __cell():
            cell = tf.contrib.rnn.GRUBlockCellV2(n_units, name='cell', reuse=reuse)
            if is_training:
                cell = tf.nn.rnn_cell.DropoutWrapper(cell=cell, input_keep_prob=1.0-drop_i, output_keep_prob=1.0-drop_o, variational_recurrent=True, input_size=inputs.shape[-1], dtype=tf.float32)
            return cell
        outputs, state = tf.nn.bidirectional_dynamic_rnn(__cell(), __cell(), inputs, seq_lens, time_major=False, dtype=tf.float32)
        outputs = tf.concat(outputs, axis=-1)
        s = tf.shape(outputs)
        W = tf.get_variable(name='W', shape=(2 * n_units, n_classes), initializer=tf.glorot_uniform_initializer(), trainable=True)
        b = tf.get_variable(name='b', shape=(n_classes, ), initializer=tf.zeros_initializer(), trainable=True)
        outputs = tf.reshape(outputs, (s[0] * s[1], s[2]), name='before_proj')
        outputs = tf.nn.xw_plus_b(outputs, W, b)
        outputs = tf.reshape(outputs, (s[0], s[1], n_classes), name='after_proj')
        log_likelihood, transition_params = tf.contrib.crf.crf_log_likelihood(
            inputs=outputs,
            tag_indices=labels,
            sequence_lengths=seq_lens
        )
        loss = tf.reduce_mean(-log_likelihood)
        decode_tags, best_scores = tf.contrib.crf.crf_decode(
            potentials=outputs,
            transition_params=transition_params,
            sequence_length=seq_lens
        )
        mask = tf.sequence_mask(seq_lens, dtype=tf.float32)
        acc = tf.reduce_sum(tf.to_float(tf.equal(decode_tags, labels)) * mask) / tf.reduce_sum(mask)
    return outputs, loss, transition_params, decode_tags, best_scores, acc

In [4]:
train_embed = {}
train_pos = {}
train_ner = {}
for p, _, files in os.walk('VLSP/train'):
    for fn in tqdm([x for x in files if x.endswith('.npy')]):
        with open(os.path.join(p, fn), 'rb') as inp:
            arr = np.load(inp)
        idx = int(fn[:-5])
        if fn[-5] == 'e':
            train_embed[idx] = arr
        elif fn[-5] == 'n':
            train_ner[idx] = [ner2idx[x] for x in arr]
        elif fn[-5] == 'p':
            train_pos[idx] = [pos2idx[x] for x in arr]
        else:
            raise ValueError()

100%|██████████| 50577/50577 [00:14<00:00, 3607.91it/s]


In [14]:
test_embed = {}
test_pos = {}
test_ner = {}
for p, _, files in os.walk('VLSP/test'):
    for fn in tqdm([x for x in files if x.endswith('.npy')]):
        with open(os.path.join(p, fn), 'rb') as inp:
            arr = np.load(inp)
        idx = int(fn[:-5])
        if fn[-5] == 'e':
            test_embed[idx] = arr
        elif fn[-5] == 'n':
            test_ner[idx] = [ner2idx[x] for x in arr]
        elif fn[-5] == 'p':
            test_pos[idx] = [pos2idx[x] for x in arr]
        else:
            raise ValueError()

100%|██████████| 8490/8490 [00:02<00:00, 3735.99it/s]


In [6]:
def pad_sequences(sequences):
    maxlen = max(len(x) for x in sequences)
#     print(np.argmax([len(x) for x in sequences]))
    if isinstance(sequences[0], np.ndarray):
        arr = np.zeros((maxlen, len(sequences)) + sequences[0].shape[1:])
    else:
        arr = np.zeros((maxlen, len(sequences)))
#     print(arr.shape)
    for i, x in enumerate(sequences):
        arr[:len(x), i] = x
    return np.transpose(arr, axes=[1, 0] + [x for x in range(2, arr.ndim)])
def get_batch(data, batch_size, shuffle=True):
    idx = np.arange(0, len(data))
    data = np.array(data)
    if shuffle:
        idx = np.random.permutation(idx)
    for i in range(0, len(idx), batch_size):
        indices = idx[i:i+batch_size]
#         print(indices)
        batch = data[indices]
        embed = pad_sequences([x[0] for x in batch])
        batch_pos = pad_sequences(np.array([x[1] for x in batch]))
        batch_ner = pad_sequences(np.array([x[2] for x in batch]))
        seq_len = [len(x[0]) for x in batch]
        yield embed, batch_pos, batch_ner, seq_len

In [15]:
train_data = [(train_embed[k], train_pos[k], train_ner[k]) for k in train_embed]
test_data = [(test_embed[k], test_pos[k], test_ner[k]) for k in test_embed]

In [35]:
tf.reset_default_graph()
session = tf.Session()
x = tf.placeholder(dtype=tf.float32, shape=(None, None, 1024, 4), name='x')
y = tf.placeholder(dtype=tf.int32, shape=(None, None), name='y')
seq_len = tf.placeholder(dtype=tf.int32, shape=(None,), name='seq_len')
drop_i = tf.placeholder(dtype=tf.float32, shape=(), name='drop_i')
drop_o = tf.placeholder(dtype=tf.float32, shape=(), name='drop_o')
elmo, elmo_l2_reg = elmo_embedding(x)
outputs, loss, transition_params, decode_tags, best_scores, acc = tagger(elmo, y, seq_len, 200, len(pos2idx), drop_i, drop_o)
optimizer = tf.train.AdamOptimizer(1e-3)
grads, _vars = zip(*optimizer.compute_gradients(loss + elmo_l2_reg))
grads, _ = tf.clip_by_global_norm(grads, clip_norm=1.0)
global_step = tf.Variable(0, name="global_step", trainable=False)
train_op = optimizer.apply_gradients(
    zip(grads, _vars),
    global_step=global_step
)
session.run(tf.global_variables_initializer())

/root/anaconda2/envs/tf19/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [36]:
for i in range(50):
    gen = get_batch(train_data, 128)
    for em, pos, _, sl in gen:
        _, s, l, a = session.run([train_op, global_step, loss, acc], feed_dict={
            x: em, y: pos, seq_len: sl, drop_i: 0.5, drop_o: 0.5
        })
        print('Step {}: loss {}, acc {}'.format(s, l, a))

Step 1: loss 88.59705352783203, acc 0.03426124155521393
Step 2: loss 81.1783447265625, acc 0.08506787568330765
Step 3: loss 79.83724975585938, acc 0.1004159227013588
Step 4: loss 65.77976989746094, acc 0.2588198482990265
Step 5: loss 63.63581466674805, acc 0.3463526666164398
Step 6: loss 57.574859619140625, acc 0.4021393358707428
Step 7: loss 51.22737121582031, acc 0.4300944209098816
Step 8: loss 52.57745361328125, acc 0.3929225206375122
Step 9: loss 46.1411018371582, acc 0.4515831470489502
Step 10: loss 43.661563873291016, acc 0.46860209107398987
Step 11: loss 47.65724182128906, acc 0.4667046070098877
Step 12: loss 40.32735061645508, acc 0.5317266583442688
Step 13: loss 42.33935546875, acc 0.5041079521179199
Step 14: loss 50.08603286743164, acc 0.46483752131462097
Step 15: loss 41.19609832763672, acc 0.4524669051170349
Step 16: loss 38.84624099731445, acc 0.5275379419326782
Step 17: loss 37.60956954956055, acc 0.5688589215278625
Step 18: loss 37.44530487060547, acc 0.5506477952003479


Step 146: loss 12.456567764282227, acc 0.8566703200340271
Step 147: loss 11.37276554107666, acc 0.8612782955169678
Step 148: loss 9.751526832580566, acc 0.8668076395988464
Step 149: loss 10.887175559997559, acc 0.8587430119514465
Step 150: loss 11.502710342407227, acc 0.8541666865348816
Step 151: loss 12.861878395080566, acc 0.8514482975006104
Step 152: loss 10.910213470458984, acc 0.8602502942085266
Step 153: loss 9.75478744506836, acc 0.8630719184875488
Step 154: loss 9.99766731262207, acc 0.8661393523216248
Step 155: loss 9.84383773803711, acc 0.8668520450592041
Step 156: loss 10.790763854980469, acc 0.8537549376487732
Step 157: loss 11.397049903869629, acc 0.8584877252578735
Step 158: loss 11.347055435180664, acc 0.876228392124176
Step 159: loss 10.475184440612793, acc 0.8614916205406189
Step 160: loss 10.411249160766602, acc 0.8609674572944641
Step 161: loss 9.532195091247559, acc 0.8687246441841125
Step 162: loss 10.261724472045898, acc 0.8768328428268433
Step 163: loss 10.171723

Step 290: loss 8.447334289550781, acc 0.8915039896965027
Step 291: loss 8.360382080078125, acc 0.893159806728363
Step 292: loss 9.895418167114258, acc 0.8744903802871704
Step 293: loss 8.82944107055664, acc 0.8903846144676208
Step 294: loss 8.511834144592285, acc 0.8991379141807556
Step 295: loss 9.210590362548828, acc 0.8742254972457886
Step 296: loss 7.8725714683532715, acc 0.8824508786201477
Step 297: loss 7.732179641723633, acc 0.8968472480773926
Step 298: loss 9.63743782043457, acc 0.8851407766342163
Step 299: loss 7.721440315246582, acc 0.8955909609794617
Step 300: loss 9.988279342651367, acc 0.8772271871566772
Step 301: loss 7.78179931640625, acc 0.9075579643249512
Step 302: loss 8.928213119506836, acc 0.8942615985870361
Step 303: loss 8.428523063659668, acc 0.8896651864051819
Step 304: loss 8.945405960083008, acc 0.8784688711166382
Step 305: loss 7.422905921936035, acc 0.8979591727256775
Step 306: loss 7.825822353363037, acc 0.8949286341667175
Step 307: loss 9.089608192443848, 

Step 435: loss 7.424810409545898, acc 0.898710310459137
Step 436: loss 7.671558380126953, acc 0.9099414944648743
Step 437: loss 8.006596565246582, acc 0.8943560123443604
Step 438: loss 7.132916450500488, acc 0.9106082320213318
Step 439: loss 7.8780083656311035, acc 0.8991644978523254
Step 440: loss 6.776481628417969, acc 0.9112163186073303
Step 441: loss 8.80131721496582, acc 0.9015915393829346
Step 442: loss 8.31631088256836, acc 0.8946162462234497
Step 443: loss 7.646773815155029, acc 0.9109125137329102
Step 444: loss 7.678265571594238, acc 0.9088593125343323
Step 445: loss 7.2358503341674805, acc 0.9039812684059143
Step 446: loss 7.617371559143066, acc 0.8999106287956238
Step 447: loss 8.108919143676758, acc 0.8980438709259033
Step 448: loss 8.255693435668945, acc 0.8979531526565552
Step 449: loss 7.1439080238342285, acc 0.9013583660125732
Step 450: loss 6.993390083312988, acc 0.9088518619537354
Step 451: loss 7.67726993560791, acc 0.902496337890625
Step 452: loss 8.667932510375977,

Step 580: loss 6.930531978607178, acc 0.9099512696266174
Step 581: loss 7.989187240600586, acc 0.9036874175071716
Step 582: loss 6.137679100036621, acc 0.9160351753234863
Step 583: loss 6.862613201141357, acc 0.907010018825531
Step 584: loss 6.509214401245117, acc 0.9169725775718689
Step 585: loss 7.279053211212158, acc 0.9035966992378235
Step 586: loss 6.338958740234375, acc 0.9165704250335693
Step 587: loss 6.371760368347168, acc 0.91190105676651
Step 588: loss 6.342034339904785, acc 0.9176719188690186
Step 589: loss 7.599836349487305, acc 0.8886809945106506
Step 590: loss 6.592465877532959, acc 0.9171403050422668
Step 591: loss 7.395898818969727, acc 0.9022101163864136
Step 592: loss 6.51641845703125, acc 0.9147214293479919
Step 593: loss 7.06380558013916, acc 0.8992537260055542
Step 594: loss 6.685604572296143, acc 0.9063344597816467
Step 595: loss 7.910696029663086, acc 0.8999713659286499
Step 596: loss 7.707444667816162, acc 0.9006859660148621
Step 597: loss 6.519501686096191, ac

...

Step 5954: loss 1.7506462335586548, acc 0.977393627166748
Step 5955: loss 2.0895607471466064, acc 0.9661529064178467
Step 5956: loss 1.838393211364746, acc 0.9655585289001465
Step 5957: loss 2.057013988494873, acc 0.9672225713729858
Step 5958: loss 1.8410885334014893, acc 0.9682299494743347
Step 5959: loss 2.164062261581421, acc 0.9694592356681824
Step 5960: loss 1.96921968460083, acc 0.9724040031433105
Step 5961: loss 1.785383939743042, acc 0.9735963344573975
Step 5962: loss 1.8510684967041016, acc 0.9719681143760681
Step 5963: loss 1.9031519889831543, acc 0.967082142829895
Step 5964: loss 1.9540729522705078, acc 0.9696789383888245
Step 5965: loss 2.031627655029297, acc 0.9657422304153442
Step 5966: loss 1.9703048467636108, acc 0.9672853350639343
Step 5967: loss 1.8099534511566162, acc 0.9728353023529053
Step 5968: loss 2.2692041397094727, acc 0.9649467468261719
Step 5969: loss 1.6935925483703613, acc 0.9730256795883179
Step 5970: loss 2.1636056900024414, acc 0.962576687335968
Step 59

Step 6094: loss 1.907570719718933, acc 0.9724847078323364
Step 6095: loss 2.409058094024658, acc 0.9639266729354858
Step 6096: loss 1.9439243078231812, acc 0.9689162969589233
Step 6097: loss 1.6759815216064453, acc 0.9729886651039124
Step 6098: loss 2.034733772277832, acc 0.9663446545600891
Step 6099: loss 1.6245241165161133, acc 0.9753531813621521
Step 6100: loss 1.711761713027954, acc 0.9674846529960632
Step 6101: loss 2.188061237335205, acc 0.9715548157691956
Step 6102: loss 1.9538161754608154, acc 0.971222996711731
Step 6103: loss 1.811983346939087, acc 0.9718654155731201
Step 6104: loss 1.88533353805542, acc 0.9749855995178223
Step 6105: loss 2.3503520488739014, acc 0.9665284156799316
Step 6106: loss 1.6532034873962402, acc 0.9752596020698547
Step 6107: loss 2.1963272094726562, acc 0.9706138968467712
Step 6108: loss 1.68248450756073, acc 0.9722834825515747
Step 6109: loss 2.148507595062256, acc 0.9691085815429688
Step 6110: loss 2.28415584564209, acc 0.9644611477851868
Step 6111: 

Step 6234: loss 1.8030601739883423, acc 0.9721473455429077
Step 6235: loss 1.8286778926849365, acc 0.9683076739311218
Step 6236: loss 2.132061243057251, acc 0.9678129553794861
Step 6237: loss 1.9023494720458984, acc 0.9728803038597107
Step 6238: loss 2.03725266456604, acc 0.9718804955482483
Step 6239: loss 1.6549736261367798, acc 0.9765856266021729
Step 6240: loss 2.2842087745666504, acc 0.9649338722229004
Step 6241: loss 2.2158308029174805, acc 0.9654357433319092
Step 6242: loss 1.8636449575424194, acc 0.9689403176307678
Step 6243: loss 2.0236401557922363, acc 0.9736341238021851
Step 6244: loss 1.7840603590011597, acc 0.9746312499046326
Step 6245: loss 1.9925258159637451, acc 0.9683817028999329
Step 6246: loss 1.8947147130966187, acc 0.971796452999115
Step 6247: loss 2.7170495986938477, acc 0.9565701484680176
Step 6248: loss 1.7040613889694214, acc 0.9756326079368591
Step 6249: loss 1.6535964012145996, acc 0.9728087186813354
Step 6250: loss 1.9568026065826416, acc 0.9705795645713806
S

Step 6374: loss 1.7275460958480835, acc 0.975049614906311
Step 6375: loss 1.5944082736968994, acc 0.9755312204360962
Step 6376: loss 1.6898938417434692, acc 0.9752763509750366
Step 6377: loss 2.1006877422332764, acc 0.973714292049408
Step 6378: loss 1.8210546970367432, acc 0.9726027250289917
Step 6379: loss 1.7448832988739014, acc 0.9743515253067017
Step 6380: loss 1.9367581605911255, acc 0.9722884297370911
Step 6381: loss 1.8973593711853027, acc 0.9746125340461731
Step 6382: loss 1.8286147117614746, acc 0.9680656790733337
Step 6383: loss 2.1454732418060303, acc 0.9645411968231201
Step 6384: loss 1.5411226749420166, acc 0.976243257522583
Step 6385: loss 1.7077839374542236, acc 0.9713459014892578
Step 6386: loss 1.7166250944137573, acc 0.9751552939414978
Step 6387: loss 2.0982789993286133, acc 0.9654247164726257
Step 6388: loss 2.4575676918029785, acc 0.9675578474998474
Step 6389: loss 1.9293839931488037, acc 0.9728056192398071
Step 6390: loss 2.0225634574890137, acc 0.9712808728218079


Step 6514: loss 1.8026773929595947, acc 0.9718849658966064
Step 6515: loss 1.8063793182373047, acc 0.9676864743232727
Step 6516: loss 1.576533317565918, acc 0.9765504598617554
Step 6517: loss 2.043117046356201, acc 0.9662120342254639
Step 6518: loss 1.8516744375228882, acc 0.9686199426651001
Step 6519: loss 2.0538578033447266, acc 0.970005989074707
Step 6520: loss 2.18243670463562, acc 0.9675090312957764
Step 6521: loss 2.0985357761383057, acc 0.9708917737007141
Step 6522: loss 1.6685048341751099, acc 0.9745834469795227
Step 6523: loss 1.8434438705444336, acc 0.9709153175354004
Step 6524: loss 2.169870138168335, acc 0.9686285257339478
Step 6525: loss 1.7359659671783447, acc 0.9712184071540833
Step 6526: loss 2.11511492729187, acc 0.9668335318565369
Step 6527: loss 2.0056042671203613, acc 0.966513454914093
Step 6528: loss 2.152495861053467, acc 0.9640610814094543
Step 6529: loss 1.7621009349822998, acc 0.9664276242256165
Step 6530: loss 2.2150650024414062, acc 0.9669030904769897
Step 65

In [37]:
total_loss = 0.0
total_acc = 0.0
sum_sl = 0
gen = get_batch(test_data, 128, False)
for em, pos, _, sl in gen:
    l, a = session.run([loss, acc], feed_dict={
        x: em, y: pos, seq_len: sl, drop_i: 0.0, drop_o: 0.0
    })
    total_acc += a * sum(sl)
    total_loss += l * sum(sl)
    sum_sl += sum(sl)
    print('Loss {}, acc {}'.format(l, a))
print('Total loss {}, total acc {}'.format(total_loss / sum_sl, total_acc / sum_sl))

Loss 6.931240081787109, acc 0.9343775510787964
Loss 5.980372428894043, acc 0.9347466826438904
Loss 8.209341049194336, acc 0.9209509491920471
Loss 7.520585536956787, acc 0.9284457564353943
Loss 8.574029922485352, acc 0.9258391857147217
Loss 7.088094711303711, acc 0.9257485270500183
Loss 8.209497451782227, acc 0.9191069602966309
Loss 6.646515846252441, acc 0.9409317970275879
Loss 7.928154945373535, acc 0.9302387237548828
Loss 9.176990509033203, acc 0.9277108311653137
Loss 9.195226669311523, acc 0.9163772463798523
Loss 7.192975044250488, acc 0.9266955256462097
Loss 8.232357025146484, acc 0.9264943599700928
Loss 6.9835100173950195, acc 0.9298391342163086
Loss 6.377013206481934, acc 0.935251772403717
Loss 7.229599475860596, acc 0.9294052124023438
Loss 6.30699348449707, acc 0.9380580186843872
Loss 7.197957992553711, acc 0.9280657172203064
Loss 7.570253372192383, acc 0.9240683913230896
Loss 6.920961380004883, acc 0.9321650862693787
Loss 8.537338256835938, acc 0.9168328046798706
Loss 8.5480365